In [21]:
import json
import pandas as pd

In [22]:
df = pd.read_csv('kaylin_new_collections.csv')
df.rename(columns={"Hierarchy in 'Science Knowledge Sources'": "hierarchy", "Label in 'Science Knowledge Sources'": "label"}, inplace=True)

def generate_tree_root(row):
    return f"/{row['Division']}/{row['hierarchy']}/{row['label']}"

def generate_json(row):
    return {
        "model": "sde_collections.collection",
        "fields": {
            "name": row["Name of Resource"],
            "config_folder": row["Machine Name"],
            "url": row["URL"],
            "turned_on": True,
            "division": row["Division"],
            "source": 2,
            "tree_root": row['tree_root'],
            "update_frequency": 2,
            "delete": False,
            "audit_hierarchy": "",
            "audit_url": "",
            "audit_mapping": "",
            "audit_label": "",
            "audit_query": "",
            "audit_duplicate_results": "",
            "audit_metrics": "",
            "cleaning_assigned_to": "",
            "notes": "",
            "new_collection": True,
        }
    }

df['tree_root'] = df.apply(generate_tree_root, axis=1)

# replace division names with ids
division_mapper = {
    "Planetary Science": 5,
    "Earth Science": 3,
    "Biological and Physical Sciences": 2,
    "Heliophysics": 4,
    "Astrophysics": 1,
}
df['Division'].replace(division_mapper, inplace=True)
df['json'] = df.apply(generate_json, axis=1)

In [23]:
with open('../sde_collections/fixtures/new_collections.json', 'w') as fixturefile:
    fixturefile.write(json.dumps(df.json.to_list(), indent=4))